In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
# 크롤링 문서 요청해서 응답객체 반환 받기


url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")


In [3]:
## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
# ul class:prdList grid4
ul=bs_obj.find("ul",{"class":"prdList grid4"})
## 품목 1개를 담고 있는 div 태그 추출
## div class:box
prd_boxes= ul.findAll("div", {"class":"box"})


In [4]:
# boxes안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로 반환하는 함수

def get_product_info(box):

    # 제품명 추출 코드
    p_tag = box.find("p",{"class":"name"})
    span = p_tag.find("span")
    
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = price_ul.findAll("span")
    
    # 최종 data 추출 후 출력
    return{"prd_name":span.text,"price":price_span[1].text,"sale_price":price_span[-1].text}
#     prd_list.append(span.text)
#     price_list.append(price_span[1].text)
#     sale_price_list.append(price_span[-1].text)

In [5]:
## 함수화  작업시작.!!

In [6]:
# 전달된 url 페이지에 접근해서 해당페이지의 전체 상품 데이터를 추출 한후 각 상품마다
# get_product_info() 함수를 호출해서 각 상품에 대한 추출 정보를 받아옴-들어온 각 상품
# 정보를 리스트로 저장한 후 해당 리스트 반환

def get_page_product(url):

    url=url              
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")

    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid4"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes= ul.findAll("div", {"class":"box"}) # 1개 페이지의 전체 상품
    
    prod_info_list=[get_product_info(box) for box in prd_boxes ] 
    # 현재 함수를 통해 접속한 문서(url인수)의
    # 각 화장품 정보가 저장되어 있는 리스트
    
    return prod_info_list

In [7]:
# 2번 페이지~!!
url="http://jolse.com/category/toners-mists/1019/?page=2"
print(get_page_product(url))

[{'prd_name': 'B_LAB Matcha Hydrating Facial Toner 150ml', 'price': 'USD 17.00', 'sale_price': 'USD 11.90'}, {'prd_name': 'META FORET 12PM Calming Toner 150ml', 'price': 'USD 13.20', 'sale_price': 'USD 10.56'}, {'prd_name': 'LOOKS&MEII Rich Routine Balance Toner 150ml', 'price': 'USD 21.00', 'sale_price': 'USD 14.70'}, {'prd_name': 'LOOKS&MEII Fresh Routine Balance Toner 150ml', 'price': 'USD 21.00', 'sale_price': 'USD 14.70'}, {'prd_name': 'MISSHA Time Revolution Best Seller Special Set', 'price': 'USD 54.99', 'sale_price': 'USD 32.94'}, {'prd_name': 'Dr.Ceuracle Pro Balance Biotics Toner 300ml', 'price': 'USD 30.00', 'sale_price': 'USD 24.00'}, {'prd_name': 'Dr.Ceuracle Cica Regen 92 Toner 120ml', 'price': 'USD 25.00', 'sale_price': 'USD 20.00'}, {'prd_name': 'MANYO FACTORY Bifida Cica Herb Toner 210ml', 'price': 'USD 19.00', 'sale_price': 'USD 13.30'}, {'prd_name': 'MANYO FACTORY Galac Whitening Vita Toner 210ml', 'price': 'USD 19.00', 'sale_price': 'USD 13.30'}, {'prd_name': 'SOME 

In [8]:
# 여러 페이지의 화장품 정보를 추출해서 df에 저장후 csv에 저장하는 코드
url="http://jolse.com/category/toners-mists/1019/?page=" # 페이지 번호를 제외한 공통 url 문자열


df_fin=pd.DataFrame()   # 빈 df생성
for i in range(1,3):  # 임시로 1,2 페이지만
    # url 완성:page번호를 추가 i 변수 값을 활용
    urlfin = url+str(i)
    #print(urlfin)
    page_product=get_page_product(urlfin)
    df=pd.DataFrame(page_product)  # 현재 url로 접속한 페이지에 대해 수집된 제품정보를 df로생성(한개 페이지 제품)
    df_fin=pd.concat([df_fin,df],axis=0,ignore_index=True)
    
df_fin

,prd_name,price,sale_price
0,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 11.99
1,SON&PARK BEAUTY WATER 340ml,USD 29.40,USD 18.99
2,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,USD 24.00,USD 11.99
3,SOME BY MI Galactomyces Pure Vitamin C Glow To...,USD 34.00,USD 17.99
4,PURITO Centella Green Level Calming Toner 200ml,USD 25.20,USD 16.99
...,...,...,...
91,ONE THING Centella Asiatica Extract 150ml,USD 20.54,USD 12.32
92,ONE THING Houttuynia Cordata Extract 150ml,USD 20.54,USD 12.32
93,ONE THING Citrus Unshiu Fruit Extract 150ml,USD 20.54,USD 12.32
94,ONE THING Brassica Oleracea Italica Extract 150ml,USD 18.96,USD 12.32


In [11]:
# 참고
for box in prd_boxes:
    produnt_info=get_product_info(box)
    print(produnt_info)

{'prd_name': 'Isntree Hyaluronic Acid Toner 200ml (Renewal)', 'price': 'USD 19.00', 'sale_price': 'USD 11.99'}
{'prd_name': 'SON&PARK BEAUTY WATER 340ml', 'price': 'USD 29.40', 'sale_price': 'USD 18.99'}
{'prd_name': 'SOME BY MI AHA BHA PHA 30 Days Miracle Toner 150ml', 'price': 'USD 24.00', 'sale_price': 'USD 11.99'}
{'prd_name': 'SOME BY MI Galactomyces Pure Vitamin C Glow Toner 200ml', 'price': 'USD 34.00', 'sale_price': 'USD 17.99'}
{'prd_name': 'PURITO Centella Green Level Calming Toner 200ml', 'price': 'USD 25.20', 'sale_price': 'USD 16.99'}
{'prd_name': 'SON&PARK Beauty Water 60ml', 'price': 'USD 10.50', 'sale_price': 'USD 6.49'}
{'prd_name': 'Pyunkang Yul Essence Toner 100ml', 'price': 'USD 14.50', 'sale_price': 'USD 9.89'}
{'prd_name': 'secretKey Hyaluron Aqua Soft Toner 500ml', 'price': 'USD 22.00', 'sale_price': 'USD 17.48'}
{'prd_name': 'ROUND LAB 1025 Dokdo Toner 200ml', 'price': 'USD 17.00', 'sale_price': 'USD 11.99'}
{'prd_name': 'SOME BY MI Snail Truecica Miracle Repair

In [9]:
df_fin.to_csv("./crawl_data/prod0805all.csv")

#### 연습문제

1. --- 품목 상세보기 링크도 추가해서 크롤링
2. --- 사이트내에서 전체페이지를 확인할 수 있는 태그가 있는지 검사하고 해당 태그에서 총 페이지를 추출한 후

    --- 전체 페이지를 크롤링 할 수 있는 프로그램으로 변경


In [12]:
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
bs_obj = BeautifulSoup(htmls,"html.parser")

In [13]:
ul=bs_obj.find("ul",{"class":"prdList grid4"})
prd_boxes= ul.findAll("div", {"class":"box"})

prd_boxes

[<div class="box">
 <span class="label-best">BEST</span>
 <span class="label-new">NEW</span>
 <div class="thumbnail">
 <span class="chk"><input class="ProductCompareClass xECPCNO_22560" type="checkbox"/></span>
 <a df-data-rolloverimg1="//jolse.com/web/product/medium/202105/cdb5a8adabd1a8b482e9d20f1336eb5b.jpg" df-data-rolloverimg2="//jolse.com/web/product/tiny/202105/574cd7fecc32349641467fa4711608d3.jpg" href="/product/detail.html?product_no=22560&amp;cate_no=1019&amp;display_group=1" name="anchorBoxName_22560"><img alt="Isntree Hyaluronic Acid Toner 200ml" class="thumb" id="eListPrdImage22560_1" src="//jolse.com/web/product/medium/202105/cdb5a8adabd1a8b482e9d20f1336eb5b.jpg"/><!-- 일반목록꾸미기아이콘 --><!-- 검색목록꾸미기아이콘 --></a>
 <div class="likeButton displaynone">
 <button type="button"><strong class="displaynone"></strong></button>
 <span class="bg-layer1"></span><span class="bg-layer2"></span><span class="bg-layer3"></span><span class="bg-layer4"></span>
 </div>
 <!--<span class="discountra

In [ ]:
https://jolse.com/product/some-by-mi-aha-bha-pha-30-days-miracle-toner-150ml/16241/?cate_no=1019&display_group=1
    https://jolse.com/category/skincare/1019/